In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import pandas as pd
from sklearn.metrics import classification_report

load the pre-trained RoBERTa model and its tokenizer. The `AutoTokenizer` and `AutoModelForSequenceClassification` classes from the `transformers` library are used to load the model, which is fine-tuned for sentiment analysis on social media text.


In [1]:

# Load RoBERTa Sentiment Analysis Model
roberta_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)
roberta_model.eval()


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

The function `analyze_roberta_sentiment()` takes text input, tokenizes it using the RoBERTa tokenizer, and passes it through the model to get sentiment predictions. 
classify the text as either "Positive" or "Negative" based on the probability threshold for the positive class.


In [2]:
def analyze_roberta_sentiment(text, tokenizer, model, threshold=0.5):
    """
    Analyze sentiment using RoBERTa and classify as Positive, or Negative.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    with torch.no_grad():
        outputs = roberta_model(**inputs)
        logits = outputs.logits

    probs = softmax(logits, dim=-1).numpy()[0]  # Convert logits to probabilities
    negative_prob, neutral_prob, positive_prob = probs  # Extract individual probabilities

    # Classify based on confidence threshold
    if positive_prob >= threshold:
        return "Positive"
    else:
        return "Negative"


load the dataset containing feedback and the true sentiment labels. The sentiment analysis function is applied to the feedback texts, and the predictions are stored in a new column. 
Finally, print a classification report to evaluate the performance of the RoBERTa model.


In [4]:

# Load Data
df = pd.read_csv(r"C:\Users\291688\Downloads\Sentiment_Analysis_NLP_Model\channel_partner_feedback.csv")  # Ensure it has 'text' and 'true_label' columns

# Analyze Sentiment for RoBERTa
df["roberta_model_prediction"] = df["Feedback_Text"].apply(lambda x: analyze_roberta_sentiment(x, tokenizer, roberta_model))

# Print classification report
print("\n RoBERTa Model Performance:")
print(classification_report(df["Sentiment"], df["roberta_model_prediction"]))



 RoBERTa Model Performance:
              precision    recall  f1-score   support

    Negative       0.89      1.00      0.94       500
    Positive       1.00      0.88      0.93       500

    accuracy                           0.94      1000
   macro avg       0.94      0.94      0.94      1000
weighted avg       0.94      0.94      0.94      1000



In [5]:
df

,Feedback_ID,Feedback_Text,Sentiment,roberta_model_prediction
0,4,"The product is well-engineered, leading to hig...",Positive,Positive
1,819,"Shipping partners are unreliable, causing delays.",Negative,Negative
2,621,Customer service representatives lack technica...,Negative,Negative
3,850,"The platform's navigation is unintuitive, maki...",Negative,Negative
4,201,Promotional discounts have driven more custome...,Positive,Positive
...,...,...,...,...
995,289,Customer demand has been steadily increasing f...,Positive,Positive
996,806,Support team is difficult to reach during crit...,Negative,Negative
997,721,Mobile notifications are excessive and cannot ...,Negative,Negative
998,57,The company maintains transparency in all tran...,Positive,Negative
